In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root directory: {project_root}")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import torch
from models.pinn import PINN
from training.trainer import Trainer
from data.dataset import get_dataloader
from utils.config import Config

Project root directory: /


ModuleNotFoundError: No module named 'models'

In [ ]:
cfg = Config(
    run_name="test_run", 
    hidden_dims=[64, 64],  
    input_dim=1,
    output_dim=2,
    residual_type="lagrangian",
    t_max=5, t_min=0

)
model = PINN(cfg)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr)
train_loader, val_loader = get_dataloader(
    data_path="data/raw/trajectory_000.npz",
    parameters_path="data/raw/parameters.json",
    config=cfg
)

In [ ]:
trainer = Trainer(
    model=model,
    config=cfg,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer
)

In [ ]:
trainer.train()

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'runs\\test_run'